<a href="https://colab.research.google.com/github/suchy1713/InteligencjaObliczeniowa/blob/master/breast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROJEKT 1

piersi  
las  
skuteczność

pliczek (breast-cancer.data): https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/

TODO:
1. <s>przetwarzanie danych</s>
2. trening modelu
3. tuning parametrów (grid search + cross val) (przynajmniej dwa) (https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/)
4. confusion matrix wypisać
5. porównać wyniki gołych danych, znormalizowanych, zestandaryzowanych i przepuszczonych przez PCA z dwiema różnymi ilościami komponentów (5 różnych testów)

In [15]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', 500)

features = ['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat']

def missing_vals(df):
    cols = df.isna().sum()/df.shape[0]
    cols = pd.DataFrame({'Column name': cols.index, 'Missing %': cols.values})
    rows = (df.isna().sum(axis=1)/df.shape[1]).value_counts()
    rows = pd.DataFrame({'Missing %': rows.index, 'Quantity': rows.values})
    print('Columns:')
    print(cols, '\n')
    print('Rows: ')
    print(rows, '\n\n')

def categorical_features(df, cols):
    for c in cols:
        print(c)
        print(df[c].value_counts(), '\n')
    print('\n\n')

def get_data():
    #wczytuje dane
    df = pd.read_csv('breast-cancer.data', header=None)
    df.columns = features

    #sprawdzam czy są brakujące wartości, bo w tym zbiorze miały być
    missing_vals(df)
    #plot twist: nie ma

    #patrze jeszcze czy nie ma błędów w zbiorze (np w komórce może być '3' i '3.0', co będzie traktowane jako inne rzeczy, a koresponduje z tą samą wartością) + też brakujące wartości mogły zostać zakodowane jakoś inaczej, tak że isna() ich nie widzi
    categorical_features(df, features)
    #plot twist v2: jest znak zapytania w niektórych xddd

    #wymieniam ? na modę z odpowiadających kolumn, sklearn zwróci mi numpy array, więc konwertuje se z powrotem do dataframea
    imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
    imputed_df = pd.DataFrame(imputer.fit_transform(df))
    imputed_df.columns = df.columns
    imputed_df.index = df.index
    df = imputed_df

    #patrze jeszcze czy sie udało
    categorical_features(df, ['node-caps', 'breast-quad'])

    #dziele sobie, bedziemy przewidywać y na podstawie X
    X, y = df[features[1:]], df[features[0]]

    #nazywam kolumny zeby sie fajniej patrzylo
    X.columns = features[1:]
    y.columns = features[0]

    #rozdzielam zmienne kategoryczne
    X = pd.get_dummies(X, columns=features[1:], drop_first=True)
    y = pd.get_dummies(y, columns=features[0])

    #biore druga kolumne z y tak zeby 1 oznaczalo nawrot, 0 brak nawrotu
    y = y['recurrence-events']

    #zamieniam na numpyowe arraye
    X, y = X.values, y.values

    return X, y

X, y = get_data()

Columns:
   Column name  Missing %
0        class        0.0
1          age        0.0
2    menopause        0.0
3   tumor-size        0.0
4    inv-nodes        0.0
5    node-caps        0.0
6    deg-malig        0.0
7       breast        0.0
8  breast-quad        0.0
9     irradiat        0.0 

Rows: 
   Missing %  Quantity
0        0.0       286 


class
no-recurrence-events    201
recurrence-events        85
Name: class, dtype: int64 

age
50-59    96
40-49    90
60-69    57
30-39    36
70-79     6
20-29     1
Name: age, dtype: int64 

menopause
premeno    150
ge40       129
lt40         7
Name: menopause, dtype: int64 

tumor-size
30-34    60
25-29    54
20-24    50
15-19    30
10-14    28
40-44    22
35-39    19
50-54     8
0-4       8
5-9       4
45-49     3
Name: tumor-size, dtype: int64 

inv-nodes
0-2      213
3-5       36
6-8       17
9-11      10
15-17      6
12-14      3
24-26      1
Name: inv-nodes, dtype: int64 

node-caps
no     222
yes     56
?        8
Name: node-caps,

In [6]:
print(X.shape)

(286, 32)


In [11]:
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA

def standardize(X, y):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

def normaliz(X, y):
    return normalize(X), y

def pca(X, y, c):
    pca = PCA()
    pca_result = pca.fit_transform(X)[:, :c]

    return pca_result, y

#print(X, y)
#print(standardize(X, y))
#print(normaliz(X, y))
#print(pca(X, y, 16))
print(pca(X, y, 8)[0])

[[-0.27579361 -0.75027339  0.93041086 ...  0.26516399  0.46672102
  -0.31537804]
 [ 1.25210462  0.13930416 -0.31112954 ... -0.53412624  0.44349453
   0.52645065]
 [ 0.87976679 -0.98239701 -0.24588068 ...  0.17063607 -0.11737724
   0.85198719]
 ...
 [-0.37093425  1.0537837  -0.53580508 ... -0.23694396 -0.30213028
   0.46745581]
 [-0.40860537 -0.57428331  0.90879992 ...  0.02230158  0.54218542
   0.49480875]
 [-1.23652407 -0.53766826  0.51360142 ...  0.00372073  0.53707152
   0.37071911]]


In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def grid_search(n_estimators, max_depths, sets, sets_names, verbose=1):
    best_acc = -np.inf
    best_ne, best_md, best_set = 0, 0, 0
    n = len(n_estimators) * len(max_depths) * len(sets_names)
    i = 0

    for ne in n_estimators:
        for md in max_depths:
            curr_set = 0
            for set_xy in sets:
                brain = RandomForestClassifier(max_depth=md, n_jobs=-1,max_features='sqrt', n_estimators=ne, oob_score=True, random_state=20) 
                score = np.mean(cross_val_score(brain, set_xy[0], set_xy[1], cv=5))

                if score > best_acc:
                    best_acc = score
                    best_ne = ne
                    best_md = md
                    best_set = curr_set

                if verbose > 0:
                    print('*** TEST ', i+1, '/', n, ' ***')
                    print('Accuracy: ', score, '\tn-estimators: ', ne, '\tmax_depth: ', md, '\tset: ', sets_names[curr_set], '\n')

                curr_set += 1
                i += 1

    print('Final accuracy: ', best_acc)
    print('Final n_estimators: ', best_ne)
    print('Final max_depth: ', best_md)
    print('Final set: ', sets_names[best_set])

In [29]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth = 1, n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True, random_state=20) 
rfc.fit(X_train, y_train)

print("Base score: " + str(rfc.score(X_test, y_test)))
print("Base params: " + str(rfc.get_params()))
rfc = RandomForestClassifier(max_depth = 1, n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True, random_state=20) 

#Wybór najlepszej ilości i głębokości drzew
#param_grid = { 
#    'n_estimators': [20, 50, 100, 400, 700],
#    'max_depth': [1,5,10,15]
#}

#Ustawiam 'accuracy' jako optymalizowany parametr i ilość złożeń w walidacji krzyżowej na 10
#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10, scoring='accuracy')
#CV_rfc.fit(X_train, y_train)

n_estimators = [20, 50, 100, 400, 700]
max_depth = [1,5,10,15]
sets = [(X, y), standardize(X, y), normaliz(X, y), pca(X, y, 16), pca(X, y, 8)]
names = ['base', 'standardized', 'normalized', 'pca - 16 components', 'pca - 8 components']

grid_search(n_estimators, max_depth, sets, names)

#print("Score after GridSearchCV: " + str(CV_rfc.best_score_))
#print("Parameters " + str(CV_rfc.best_params_))

Base score: 0.6
Base params: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': -1, 'oob_score': True, 'random_state': 20, 'verbose': 0, 'warm_start': False}
*** TEST  1 / 100  ***
Accuracy:  0.7062915910465819 	n-estimators:  20 	max_depth:  1 	set:  base 

*** TEST  2 / 100  ***
Accuracy:  0.7062915910465819 	n-estimators:  20 	max_depth:  1 	set:  standardized 

*** TEST  3 / 100  ***
Accuracy:  0.6992740471869328 	n-estimators:  20 	max_depth:  1 	set:  normalized 

*** TEST  4 / 100  ***
Accuracy:  0.7027828191167573 	n-estimators:  20 	max_depth:  1 	set:  pca - 16 components 

*** TEST  5 / 100  ***
Accuracy:  0.6922565033272836 	n-estimators:  20 	max_depth:  1 	set:  pca - 8 components 

*** TEST  6

In [30]:
from sklearn.metrics import confusion_matrix

X, y = pca(X, y, 16)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30, random_state=42)

rfc.set_params(max_depth = 5, n_estimators = 20)
rfc.fit(X_train, y_train)
conf_mat = confusion_matrix(y_test, rfc.predict(X_test), labels=[0,1])

print("Confusion matrix")
print(conf_mat)

Confusion matrix
[[17  1]
 [ 7  5]]
